#Advanced Example
-----------------------------
---------------------------
#Idea By [Jesse Steinweg-Woods](https://jessesw.com/Data-Science-Skills/)
----------------------------------------------------

#Reference CS109
--------------------------

Scraping Data Science Skills
=======================
- What Skills are in demand for data scientist

Scraping pages of indeed.com

**All the imports are case sensitive in python**

In [8]:
import urllib
import time
import socket
from IPython.display import HTML
import bs4
import _pickle as pickle
import numpy as np
import operator
import re

import pandas as pd
from pandas import DataFrame
from pandas import Series

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

import seaborn as sns
sns.set_context("talk")
sns.set_style("white")

In [29]:
url = "https://www.indeed.co.in/jobs?q=Data+Science&l="

source = urllib.request.urlopen(url).read()
# source

bs_tree = bs4.BeautifulSoup(source)
# print(bs_tree.prettify())

job_postings = bs_tree.find(id = "searchCount").contents[0]
# job_postings

bs4.element.NavigableString

In [39]:
print("The search yeilded %s hits" % (job_postings).split()[3])
int(job_postings.split()[3].translate({ord(','): None}))

The search yeilded 15,599 hits


15599